# 自定义入门

[![下载Notebook](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_notebook.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_mid_low_level_api.ipynb)&emsp;[![下载样例代码](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_download_code.png)](https://obs.dualstack.cn-north-4.myhuaweicloud.com/mindspore-website/notebook/tutorials-develop/tutorials/zh_cn/mindspore_mid_low_level_api.py)&emsp;[![查看源文件](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/tutorials-develop/tutorials/source_zh_cn/intermediate/build_net//mid_low_level_api.ipynb)

MindSpore向用户提供了高阶、中阶和低阶3个不同层次的API，详细内容参见[基本介绍-层次结构内容章节](https://www.mindspore.cn/tutorials/zh-CN/master/beginner/introduction.html#层次结构)。

为方便控制网络的执行流程，MindSpore提供了高阶的训练和推理接口`mindspore.Model`，通过指定要训练的神经网络模型和常见的训练设置，调用`train`和`eval`方法对网络进行训练和推理。同时，用户如果想要对特定模块进行个性化设置，也可以调用对应的中低阶接口自行定义网络的训练流程。

本章将会介绍使用MindSpore提供的中低阶API自定义损失函数、优化器、训练流程、metric、自定义验证流程模块。

## 定义数据集

!!!wushuiqin，这里与入门的《数据处理-自定义数据集》内容雷同，但是代码不相同，一个用return，一个用yield的方式，跟郭志坚确认一下，到底哪种才是最佳写法，确认后把这里的内容挪到入门的《数据处理-自定义数据集》。

MindSpore的`mindspore.dataset`模块集成了常见的数据处理功能：用户既可以调用此模块的相关接口来[加载常见的数据集](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.dataset.html)，也可以构造数据集类并使用`mindspore.dataset.GeneratorDataset`接口实现自定义数据集加载。

如下示例定义`get_data`函数生成样本数据及对应的标签，定义`create_dataset`函数加载自定义数据集。

In [79]:
import numpy as np
import mindspore.dataset as ds

def get_data(data_num, data_size):
    """生成样本数据及对应的标签"""
    for _ in range(data_num):
        data = np.random.randn(data_size)
        p = np.array([1, 0, -3, 5])
        label = np.polyval(p, data).sum()
        yield data.astype(np.float32), np.array([label]).astype(np.float32)

def create_dataset(data_num, data_size, batch_size=32, repeat_size=1):
    """定义数据集"""
    input_data = ds.GeneratorDataset(list(get_data(data_num, data_size)), column_names=['data', 'label'])
    input_data = input_data.batch(batch_size)
    input_data = input_data.repeat(repeat_size)
    return input_data

## 定义网络

mindspore.nn`类是构建所有网络的基类，也是网络的基本单元。当用户需要自定义网络时，可以继承`nn.Cell`类，并重写`__init__`方法和`construct`方法。`mindspore.ops`模块提供了基础算子的实现，`nn.Cell`模块实现了对基础算子的进一步封装，用户可以根据需要，灵活使用不同的算子。

如下示例使用`nn.Cell`构建一个简单的全连接网络，用于后续自定义内容的示例片段代码。

In [ ]:
from mindspore import nn
import mindspore.common.initializer as init

class MyNet(nn.Cell):
    """定义全连接网络"""
    def __init__(self, input_size=32):
        super(MyNet, self).__init__()
        self.fc1 = nn.Dense(input_size, 120, weight_init=init.Normal(0.02))
        self.fc2 = nn.Dense(120, 84, weight_init=init.Normal(0.02))
        self.fc3 = nn.Dense(84, 1, weight_init=init.Normal(0.02))
        self.relu = nn.ReLU()

    def construct(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## 自定义损失函数

损失函数（Loss Function）用于衡量预测值与真实值差异的程度。深度学习中，模型训练就是通过不停地迭代来缩小损失函数值的过程，因此在模型训练过程中损失函数的选择非常重要，定义一个好的损失函数可以帮助损失函数值更快收敛，达到更好的精度。

[mindspore.nn](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.nn.html#id13)提供了许多通用损失函数供用户选择， 也支持用户根据需要自定义损失函数。

自定义损失函数类时，既可以继承网络的基类`nn.Cell`，也可以继承损失函数的基类`nn.LossBase`。下面将分别使用继承Cell和继承LossBase的方法来定义平均绝对误差损失函数(Mean Absolute Error，MAE)，MAE算法的公式如下所示：

$$ loss= \frac{1}{m}\sum_{i=1}^m\lvert y_i-f(x_i) \rvert $$

上式中$f(x)$为预测值，$y$为样本真实值，$loss$为预测值与真实值之间距离的平均值。

### 继承Cell定义损失函数

`nn.Cell`既可以用于定义网络，也可以定义损失函数。使用`nn.Cell`定义损失函数与定义一个普通的网络的差别在于，其执行逻辑用于计算前向网络输出与真实值之间的误差。

示例代码如下：

In [81]:
from mindspore import nn, ops

class MyMAELoss1(nn.Cell):
    """定义损失"""
    def __init__(self):
        super(MyMAELoss1, self).__init__()
        self.abs = ops.Abs()
        self.reduce_mean = ops.ReduceMean()

    def construct(self, predict, target):
        x = self.abs(predict - target)
        return self.reduce_mean(x)

### 继承LossBase定义损失函数

`nn.LossBase`作为损失函数的基类，使用其自定义损失函数时，需要重写`__init__`方法和`construct`方法，使用`get_loss`方法计算损失。示例代码如下：

In [82]:
from mindspore import nn, ops

class MyMAELoss2(nn.LossBase):
    """定义损失"""
    def __init__(self, reduction="mean"):
        super(MyMAELoss2, self).__init__(reduction)
        self.abs = ops.Abs()

    def construct(self, predict, target):
        x = self.abs(predict - target)
        return self.get_loss(x)

!!!wushuiqin，你写得这个比官网的API还要简单，有1个疑问，为什么MyMAELoss1有reduce_mean，MyMAELoss2没有，区别在哪里？要解释清楚。

!!!wushuiqin，下面是官网提供的API demo，需要提供两者的运行结果，并加上对运行结果的解释，是否符合上面公式。https://mindspore.cn/docs/api/zh-CN/master/api_python/nn/mindspore.nn.LossBase.html?highlight=get_loss#mindspore.nn.LossBase.get_loss

from mindspore import Tensor

```python
# Case 1: logits.shape = labels.shape = (3,)
logits = Tensor(np.array([1, 2, 3]), mindspore.float32)
labels = Tensor(np.array([1, 2, 2]), mindspore.float32)
output = MyNet(logits, labels)
print(output)

# Case 2: logits.shape = labels.shape = (3, 3)
logits = Tensor(np.array([[1, 2, 3],[1, 2, 3],[1, 2, 3]]), mindspore.float32)
labels = Tensor(np.array([[1, 2, 2],[1, 2, 3],[1, 2, 3]]), mindspore.float32)
output = MyNet(logits, labels)
print(output)
```

## 自定义优化器

优化器在模型训练过程中，用于计算和更新网络参数，合适的优化器可以有效减少训练时间，提高模型性能。

[mindspore.nn](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.nn.html#id14)提供了许多通用的优化器供用户选择，同时也支持用户根据需要自定义优化器。

!!!wushuiqin，下面代码MyMomentum的公式是什么？要对应解读，不能只是把md挪到ipython并跑通，要理解，然后告诉别人这是什么。

自定义优化器时可以继承优化器基类`nn.Optimizer`，重写`__init__`方法和`construct`方法实现参数的更新。下面使用基础的运算算子自定义优化器，示例代码如下：

In [83]:
from mindspore import Tensor, Parameter
from mindspore import nn, ops
from mindspore import dtype as mstype

class MyMomentum1(nn.Optimizer):
    """定义优化器"""
    def __init__(self, params, learning_rate, momentum=0.9, use_nesterov=False):
        super(MyMomentum1, self).__init__(learning_rate, params)
        self.momentum = Parameter(Tensor(momentum, mstype.float32), name="momentum")
        self.use_nesterov = use_nesterov
        self.moments = self.parameters.clone(prefix="moments", init="zeros")
        self.assign = ops.Assign()

    def construct(self, gradients):
        lr = self.get_lr()
        params = self.parameters
        for i in range(len(params)):
            self.assign(self.moments[i], self.moments[i] * self.momentum + gradients[i])
            if self.use_nesterov:
                update = params[i] - (self.moments[i] * self.momentum + gradients[i]) * lr
            else:
                update = params[i] - self.moments[i] * lr
            self.assign(params[i], update)
        return params

!!!wushuiqin, ApplyMomentum这个算子是什么来的？有什么用，为什么需要用ApplyMomentum这个算子，跟SGD这些算子you有啥区别？拿出一个简单的SGD来解释不香吗？

MindSpore也封装了`ApplyMomentum`算子供用户使用，使用`ApplyMomentum`算子自定义优化器：

In [84]:
class MyMomentum2(nn.Optimizer):
    """定义优化器"""
    def __init__(self, params, learning_rate, momentum=0.9, use_nesterov=False):
        super(MyMomentum2, self).__init__(learning_rate, params)
        self.moments = self.parameters.clone(prefix="moments", init="zeros")
        self.momentum = momentum
        self.opt = ops.ApplyMomentum(use_nesterov=use_nesterov)

    def construct(self, gradients):
        params = self.parameters
        success = None
        for param, mom, grad in zip(params, self.moments, gradients):
            success = self.opt(param, mom, self.learning_rate, grad, self.momentum)
        return success

!!!wushuiqin，下面是官网提供的API demo，需要提供两者的运行结果，并加上对运行结果的解释，是否符合上面公式。

## 自定义训练流程

`mindspore.nn`模块提供了`train`和`eval`训练和推理的接口方便用户在训练过程中使用，此外用户还可以自行定义训练过程。!!!wushuiqin,补充好下面这句话，解释清楚为什么要定义损失函数，不懂问问小贤：自定以训练过程需要首先定义损失函数，然后把损失函数和训练流程结合起来。

### 定义损失网络

定义损失网络`MyWithLossCell`，将前向网络与损失函数连接起来。!!!是否一定要自定义损失函数？如果不定义呢？能不能用上面自定义MyMAELoss1的损失函数。如果损失函数用nn接口的呢？是否就不能自定义训练流程了？

In [85]:
class MyWithLossCell(nn.Cell):
    """定义损失网络"""

    def __init__(self, backbone, loss_fn):
        super(MyWithLossCell, self).__init__(auto_prefix=False)
        self.backbone = backbone
        self.loss_fn = loss_fn

    def construct(self, data, label):
        """连接前向网络和损失函数"""
        out = self.backbone(data)
        return self.loss_fn(out, label)

    def backbone_network(self):
        """要封装的骨干网络"""
        return self.backbone

### 定义训练流程

定义训练流程`MyTrainStep`，该类继承`nn.TrainOneStepCell`，`nn.TrainOneStepCell`封装了损失网络和优化器，在执行训练时通过`ops.GradOperation`算子来进行梯度的获取，通过优化器来实现权重的更新。

In [86]:
class MyTrainStep(nn.TrainOneStepCell):
    """定义训练流程"""

    def __init__(self, network, optimizer):
        """参数初始化"""
        super(MyTrainStep, self).__init__(network, optimizer)
        self.grad = ops.GradOperation(get_by_list=True)

    def construct(self, data, label):
        """构建训练过程"""
        weights = self.weights
        loss = self.network(data, label)
        grads = self.grad(self.network, weights)(data, label)
        return loss, self.optimizer(grads)

### 执行训练：

执行训练，并打印输出每个Epoch的损失值。

In [87]:
# 生成多项式分布的训练数据
dataset_size = 32
ds_train = create_dataset(2048, dataset_size)

# 定义网络
net = MyNet()
# 损失函数
loss_func = MyMAELoss()
# 优化器
opt = MyMomentum(net.trainable_params(), 0.01)
# 构建损失网络
net_with_criterion = MyWithLossCell(net, loss_func)
# 构建训练网络
train_net = MyTrainStep(net_with_criterion, opt)

# 执行训练，每个epoch打印一次损失值
epochs = 5
for i in range(epochs):
    for train_x, train_y in ds_train:
        train_net(train_x, train_y)
        loss_val = net_with_criterion(train_x, train_y)
    print(f"Epoch:[{i+1}/{epochs}], loss:{loss_val}")

Epoch:[1/5], loss:147.29823
Epoch:[2/5], loss:16.221462
Epoch:[3/5], loss:15.118769
Epoch:[4/5], loss:17.514624
Epoch:[5/5], loss:12.88929


## 自定义评价指标

当训练任务结束，常常需要评价指标（Metrics）评估函数来评估模型的好坏。常用的评价指标混淆矩阵、准确率 Accuracy、精确率 Precision、召回率 Recall等。

[mindspore.nn](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/mindspore.nn.html#id16)模块提供了常见的评估函数，用户也可以根据需要自行定义评估指标。

!!!wushuiqin，同样的问题，下面代码MyMAE的公式是什么？要对应解读，不能只是把md挪到ipython并跑通，要理解，然后告诉别人这是什么。

示例代码如下：

In [88]:
class MyMAE(nn.Metric):
    """定义metric"""

    def __init__(self):
        super(MyMAE, self).__init__()
        self.clear()

    def clear(self):
        self.abs_error_sum = 0
        self.samples_num = 0

    def update(self, *inputs):
        y_pred = inputs[0].asnumpy()
        y = inputs[1].asnumpy()
        error_abs = np.abs(y.reshape(y_pred.shape) - y_pred)
        self.abs_error_sum += error_abs.sum()
        self.samples_num += y.shape[0]

    def eval(self):
        return self.abs_error_sum / self.samples_num

!!!wushuiqin，下面是官网提供的API demo，需要提供两者的运行结果，并加上对运行结果的解释，是否符合上面公式。

## 自定义验证流程

!!!在什么场合下需要自定义验证流程，要说明清楚， 原来用的eval也说明清楚。

mindspore.nn模块提供了评估网络包装函数[nn.WithEvalCell](https://www.mindspore.cn/docs/api/zh-CN/master/api_python/nn/mindspore.nn.WithEvalCell.html#mindspore.nn.WithEvalCell)，用户也可以自行定义评估网络包装函数：

In [89]:
class MyWithEvalCell(nn.Cell):
    """定义验证流程"""

    def __init__(self, network):
        super(MyWithEvalCell, self).__init__(auto_prefix=False)
        self.network = network

    def construct(self, data, label):
        outputs = self.network(data)
        return outputs, label

执行推理并评估：


In [90]:
# 获取验证数据
ds_eval = create_dataset(128, dataset_size, 1)
# 定义评估网络
eval_net = MyWithEvalCell(net)
eval_net.set_train(False)
# 定义评估指标
mae = MyMAE()

# 执行推理过程
for eval_x, eval_y in ds_eval:
    output, eval_y = eval_net(eval_x, eval_y)
    mae.update(output, eval_y)

mae_result = mae.eval()
print("MAE: ", mae_result)

MAE:  16.023015797138214


输出评估误差，MAE与模型在训练集上效果大致相同。